In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ROOT
def Read_Root_File(file_or_dir, rebin=1):
    if file_or_dir in [None, "", []]:
        return pd.DataFrame(), pd.DataFrame()

    if isinstance(file_or_dir, str):
        root_obj = ROOT.TFile.Open(file_or_dir)
    else:
        root_obj = file_or_dir

    keys = root_obj.GetListOfKeys()
    df_data = pd.DataFrame()
    df_statistics = pd.DataFrame()

    for key in keys:
        try:
            obj = key.ReadObj()
            if not isinstance(obj, ROOT.TH1D):
                continue

            hist = obj
            if hist.GetNbinsX() > rebin:
                hist = hist.Rebin(rebin)

            hist_name = key.GetName()
            bin_values = [hist.GetBinContent(i) for i in range(1, hist.GetNbinsX() + 1)]
            bin_centers = [hist.GetBinCenter(i) for i in range(1, hist.GetNbinsX() + 1)]
            bin_errors = [hist.GetBinError(i) for i in range(1, hist.GetNbinsX() + 1)]

            df_data = pd.concat([df_data, pd.DataFrame({
                hist_name + "_values": bin_values,
                hist_name + "_bins": bin_centers,
                hist_name + "_errors": bin_errors
            })], axis=1)

            df_statistics = pd.concat([df_statistics, pd.DataFrame([{
                "mean": hist.GetMean(),
                "std": hist.GetStdDev(),
                "entries": hist.GetEntries(),
                "title": hist.GetTitle(),
                "xlabel": hist.GetXaxis().GetTitle(),
                "ylabel": hist.GetYaxis().GetTitle(),
                "key": hist_name
            }])], ignore_index=True)

        except Exception as e:
            print(f"[ERROR] Failed to read {key.GetName()}: {e}")

    if not df_statistics.empty:
        df_statistics = df_statistics.set_index("key")

    return df_data, df_statistics

def Bin_Edges_Values_Errors(dataframe, name):
    bin_centers = dataframe[name + "_bins"].dropna().values
    bin_values = dataframe[name + "_values"].dropna().values
    bin_errors = dataframe[name + "_errors"].dropna().values

    bin_width = bin_centers[1] - bin_centers[0]
    bin_edges = bin_centers - bin_width / 2
    bin_edges = np.append(bin_edges, bin_centers[-1] + bin_width / 2)

    return bin_edges, bin_values, bin_errors


def plot_hists_stack(
    hist_name,
    description,
    signals,
    backgrounds,
    offset=0.65,
    xlabel=r"$m_H~[GeV]$",
    ylabel=r"$N_{\mathrm{Events}}$",
    xmin=0,
    xmax=1000,
    log=False,
    normalize=False,
    scale=1,
    int_lumi=109080,
    Rebin=2,
    savepath=None
):
    import numpy as np
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 8))
    stacked_values = None
    stack_handles = []
    stack_labels = []

    for bg in backgrounds:
        df, _ = Read_Root_File(bg["file"], Rebin)
        bin_edges, values, _ = Bin_Edges_Values_Errors(df, hist_name)
        values = np.array(values) * int_lumi

        if stacked_values is None:
            baseline = np.zeros_like(values)
        else:
            baseline = stacked_values

        h = ax.stairs(
            baseline + values,
            bin_edges,
            baseline=baseline,
            color=bg["color"],
            fill=True,
            edgecolor="black",
            linewidth=1.2
        )
        stacked_values = baseline + values
        stack_handles.append(h)
        stack_labels.append(bg["label"])

    signal_handles = []
    signal_labels = []

    for sig in signals:
        df, _ = Read_Root_File(sig["file"], Rebin)
        bin_edges, values, _ = Bin_Edges_Values_Errors(df, hist_name)

        if normalize:
            values = values * scale

        values = values * int_lumi

        h = ax.stairs(
            values,
            bin_edges,
            color=sig["color"],
            linestyle=sig["linestyle"],
            linewidth=2
        )
        label = sig["label"]
        if normalize:
            label += f" (×{scale})"
        signal_handles.append(h)
        signal_labels.append(label)

    # Styling
    ax.set_xlabel(xlabel, fontsize=18, loc="right")
    ax.set_ylabel(ylabel, fontsize=18, loc="top")
    ax.set_xlim(xmin, xmax)
    if log:
        ax.set_yscale("log")
    
    ax.minorticks_on()
    ax.tick_params(which='major', length=7, width=1.3, direction='in', top=True, right=True)
    ax.tick_params(which='minor', length=3, width=1.1, direction='in', top=True, right=True)

    ax.set_title(r'$\bf{CMS}$' + " " +  " " + r'$\it{Work~in~Progress}$', 
             fontsize=18, loc="left")
    ax.set_title(f"{int_lumi/1000:.2f}" + r" fb$^{-1}$, 13.6 TeV", fontsize=15, loc="right")

    ax.text(offset, 0.85, description, transform=ax.transAxes, fontsize=15, va='top', ha='left')

    handles = signal_handles + stack_handles
    labels = signal_labels + stack_labels

    leg = ax.legend(
        handles=handles,
        labels=labels,
        fontsize=12, frameon=False, loc="upper center", ncol=3)

    for line in leg.get_lines():
        line.set_linewidth(3)
        # Set proper y-limits based on stacked background and signals
    y_max_stack = np.max(stacked_values) if stacked_values is not None else 0
    y_max_signals = max([np.max(values * int_lumi * scale if normalize else values * int_lumi)
                         for sig in signals
                         for df, _ in [Read_Root_File(sig["file"], Rebin)]
                         for _, values, _ in [Bin_Edges_Values_Errors(df, hist_name)]],
                        default=0)

    y_max = max(y_max_stack, y_max_signals) * 1.2

    if not log:
        ax.set_ylim([0, y_max])
    else:
        y_min_nonzero = np.min(stacked_values[stacked_values > 0]) if stacked_values is not None else 1e-1
        ax.set_ylim([max(1e-1, y_min_nonzero), y_max])

    plt.tight_layout()

    if savepath:
        plt.savefig(savepath)
        print(f"[INFO] Saved plot to: {savepath}")
    plt.show()


In [ ]:
signals = [
    {
        "file": "/Users/artemis/desktop/hists/ZH_12.root",
        "label": r"$ZH \rightarrow 4b,~m_a=12$",
        "color": "red",
        "linestyle": "--",
    },
    {
        "file": "/Users/artemis/desktop/hists/ZH_20.root",
        "label": r"$ZH \rightarrow 4b,~m_a=20$",
        "color": "blue",
        "linestyle": "-",
    }
]

backgrounds = [
    {"file": "/Users/artemis/desktop/hists/QCD.root",    "label": "QCD",           "color": "darkred"},
    {"file": "/Users/artemis/desktop/hists/W4.root",      "label": r"$W \to \ell\nu$", "color": "pink"},
    {"file": "/Users/artemis/desktop/hists/TTbb.root",    "label": r"$t\bar{t}+b\bar{b}$", "color": "seagreen"},
    {"file": "/Users/artemis/desktop/hists/TTcc.root",    "label": r"$t\bar{t}+c\bar{c}$", "color": "mediumseagreen"},
    {"file": "/Users/artemis/desktop/hists/TTlf.root",    "label": r"$t\bar{t}+$ light",  "color": "darkseagreen"},
    {"file": "/Users/artemis/desktop/hists/Z2Nu.root",    "label": "Other Bkgs",     "color": "royalblue"},
]

plot_hists_stack(
    hist_name="mass_H_resolved",
    description="Higgs mass (boosted)",
    signals=signals,
    backgrounds=backgrounds,
    log=False,
    normalize=True,
    scale=20,
    xlabel=r"$m_H~[GeV]$",
    ylabel=r"$N_{\mathrm{Events}}$",
    xmin=0,
    xmax=1000,
    Rebin=1,
    savepath="/Users/artemis/desktop/hists/mass_H_boosted.pdf"
)

In [ ]:
#and to plot all your needed histos in the root files do like:
xlabels = {
    "mass_H_boosted": r"$m_H~[\mathrm{GeV}]$",
    "pt_H_boosted": r"$p_T^H~[\mathrm{GeV}]$",
    # Add more .....
}

In [ ]:
for hist_name, xlabel in xlabels.items():
    plot_hists_stack(
        hist_name=hist_name,
        description=hist_name.replace("_", " ").title(),
        signals=signals,
        backgrounds=backgrounds,
        log=False,
        normalize=True,
        scale=1,
        xlabel=xlabel,
        ylabel=r"$N_{\mathrm{Events}}$",
        xmin=0,
        xmax=1000,
        Rebin=1,
        savepath=f"/Users/artemis/desktop/hists/{hist_name}.pdf"
    )
